In [11]:
''' IMPORT ALL THE THINGS '''
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array, smart_resize
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
# from tensorflow.keras.applications.vgg19 import VGG19

In [12]:
''' DEFINE VARIABLES '''

CWD = os.getcwd().replace('\\','/') # get current workspace directory
IMAGE_DIR = f'{CWD}/images'
TRAIN_DATASET_DIR = f'{IMAGE_DIR}/train'
TEST_DATASET_DIR = f'{IMAGE_DIR}/test'
VAL_DATASET_DIR = f'{IMAGE_DIR}/validation'

CATEGORIES = ['clementine','grapefruit','orange']

IMAGE_RESIZE = (224, 224) # size of image to put in Model

AUGMENT_N_IMAGE = 4 # how many time each image get the augmentation

AUGMENT_SAVE_PREFIX = 'aug'

Save_model = ''


In [13]:
''' Create Data Augmentation '''
def run_augmentation( isRun = 0 ):

    if isRun:

        print('Running Augmentation...\n')

        datagen = ImageDataGenerator(
            rotation_range=45,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='reflect' # optional : nearest, constant(125), reflect, wrap
        ) 

        ''' METHOD 1 DOES NOT WORK , don't know why '''
        # aug_train = datagen.flow_from_directory(
        #     TRAIN_DATASET_DIR,
        #     target_size=IMAGE_RESIZE,
        #     class_mode='categorical',
        #     batch_size=1,
        #     save_to_dir=f"{TRAIN_DATASET_DIR}/preview",
        #     save_prefix='aug',
        #     save_format='jpeg'
        # )
        ''' METHOD 2 '''
        try:
            for category in CATEGORIES:

                train_image_in_directory = [] 

                train_category_directory = f"{TRAIN_DATASET_DIR}/{category}"

                for i, image_name in enumerate(os.listdir(train_category_directory)):
                    
                    file_path = f"{train_category_directory}/{image_name}"
                    
                    if (image_name.find(AUGMENT_SAVE_PREFIX) != -1):  # delete old augmentation
                        if (os.path.exists(file_path)):
                            os.remove(file_path)

                    elif (image_name.split('.')[1] in ['png', 'jpg', 'jpeg']):

                        # image = cv2.imread(
                        #     f'{class_directory}/{image_name}', cv2.IMREAD_UNCHANGED)       ### NOT USING OpenCV cause the image is in BGR ordering not RGB that make img BLUE
                        # image = cv2.resize(image, IMAGE_RESIZE, interpolation=cv2.INTER_AREA)

                        image = load_img(file_path)
                        image = smart_resize(image, IMAGE_RESIZE)
                        image = img_to_array(image)
                        train_image_in_directory.append(image)

                train_image_in_directory = np.array(train_image_in_directory)

                print(train_image_in_directory.shape)

                i = 0
                
                # save_to_dir (option) : f'{train_category_directory}', 'augmented'
                for batch in datagen.flow(train_image_in_directory, batch_size=1, save_to_dir=f'{train_category_directory}', save_prefix=AUGMENT_SAVE_PREFIX, save_format='jpg'):
                    i += 1
                    # datagen.flow make infinite loop NEED A MANUAL BREAK!!!
                    if i >= len(train_image_in_directory) * AUGMENT_N_IMAGE:
                        break

            print('\nAugmentation Success!!!')

        except Exception as e:

            print('\nAugmentation Failed...')
            print('Exception is\n',e)
    
    


In [14]:
run_augmentation()  

In [15]:
''' LOAD DATA into Variables '''
try:
    print("Loading data...\n")
    
    print('-----------Train-----------')
    #Load train data

    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        TRAIN_DATASET_DIR, # train_data path
        image_size=IMAGE_RESIZE, # each image resize
        batch_size=1,
        label_mode='categorical',
        color_mode='rgb',
        shuffle=True
    )

    print(train_dataset.class_names)  # class check
    print(train_dataset)


    print("-----------Validation-----------")
    #Load validation data
    valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        VAL_DATASET_DIR,  # validation_data path
        image_size=IMAGE_RESIZE,  # each image resize
        batch_size=1, 
        label_mode='categorical',
        color_mode='rgb',
        shuffle=True
    )

    print(valid_dataset.class_names)  # class check
    print(valid_dataset)

    print("\nLoad data Success!!!")
    
except Exception as e:

    print("\nLoad data failed...")
    print("Exception is",e)




Loading data...

-----------Train-----------
Found 48 files belonging to 3 classes.
['clementine', 'grapefruit', 'orange']
<BatchDataset shapes: ((None, 224, 224, 3), (None, 3)), types: (tf.float32, tf.float32)>
-----------Validation-----------
Found 22 files belonging to 3 classes.
['clementine', 'grapefruit', 'orange']
<BatchDataset shapes: ((None, 224, 224, 3), (None, 3)), types: (tf.float32, tf.float32)>

Load data Success!!!


In [16]:
''' Create a model '''

model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 3)),
                                    tf.keras.layers.MaxPool2D(2, 2),
                                    #
                                    tf.keras.layers.Conv2D(
                                        32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
                                    tf.keras.layers.MaxPool2D(2, 2),
                                    #
                                    tf.keras.layers.Conv2D(
                                        64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
                                    tf.keras.layers.MaxPool2D(2, 2),

                                    ##
                                    tf.keras.layers.Flatten(),
                                    ##
                                    tf.keras.layers.Dense(
                                        512, activation='relu'),
                                    ##
                                    tf.keras.layers.Dense(
                                        3, activation='softmax')

                                    ])


In [17]:
# model.summary() # show all the model's layer

In [18]:
''' Compiling '''

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
los = tf.keras.losses.CategoricalCrossentropy()
met = tf.keras.metrics.Accuracy()

model.compile( optimizer = opt , loss = los , metrics = met )


In [19]:
''' Training '''
history = model.fit(
    train_dataset,  # both data and label cause we use preprocessing
    validation_data=valid_dataset,
    batch_size=32,
    epochs=10, ### can be change
    shuffle=True 
)


Epoch 1/10
48/48 [==============================] - 9s 184ms/step - loss: 133.9295 - accuracy: 0.1875 - val_loss: 2.5853 - val_accuracy: 0.0000e+00
Epoch 2/10
48/48 [==============================] - 9s 180ms/step - loss: 1.2583 - accuracy: 0.0000e+00 - val_loss: 2.1313 - val_accuracy: 0.0000e+00
Epoch 3/10
25/48 [==============>...............] - ETA: 4s - loss: 0.3370 - accuracy: 0.0133

KeyboardInterrupt: 

In [ ]:
# history.history 

In [ ]:
''' Save the model '''
# model.save(f'{CWD}/Model/{Save_model}')

In [ ]:
''' Evaluating '''

' Evaluating '

In [ ]:
''' Predicting '''

' Predicting '